In [1]:
import pandas as pd
import numpy as np
import spacy
import pickle
import os
import textblob
import re
import string
from textblob import TextBlob

### loading data

In [21]:
files = os.listdir('data/authors')

In [3]:
os.getcwd()

'C:\\Users\\StepanAsus\\Documents\\UvA\\Subjects\\Master Thesis\\Code'

In [22]:
file = open('data/authors/' + files[0], 'rb')
object_file = pickle.load(file)
file.close()

In [23]:
object_file

{'---': None,
 '--------__--------': None,
 '------___--___------': ['Exommerce products specifically. ',
  '**UPVOTES FOR EVERYONE**',
  '*whoops*',
  "Considering his followers are fake, you probably won't get much help. ",
  "As a creative professional, I feel you. There's so much competition and your only good as what people can hire you for. \n\nI never had a full time job before so I became an entrepreneur by force. It sucks and that's something no one ever talks about - apparently everyone is an entrepreneur by choice. \n\nIt's all about 'I hate my job, I want to do my own thing' but you know what's worse? being *homeless*.\n\nSorry I'm going off a tangent here, I'm just saying building capital for awhile should never be looked down on because I dream that I could do that every day. ",
  'Dora the Destroya',
  'No problem. Thanks for everything you do!',
  'Then you need to step up your game, fam!',
  "First I want to apologize if I insinuated you guys weren't doing a good job. 

In [24]:
test_user = object_file['------___--___------']

In [26]:
object_file.keys()

dict_keys(['---', '--------__--------', '------___--___------', '-----BroAway-----', '----_____----', '----branflakes----', '---0---', '---Eagle---', '---Earth---', '---_--_---', '---_-o-', '--4-8-15-16-23-42', '--CAT--', '--Danger--', '--Dawn--', '--Edog--', '--Imhighrightnow--', '--Kai--', '--MrsNesbitt-', '--Noct', '--Potatoes--', '--Quartz--', '--Satan--', '--TaCo--', '--TheLady0fTheLake--', '--Void--', '--_Jesus_--', '--__--__--__--', '--__--____--__--', '--atmos', '--boobies--', '--cheese--', '--hypnos--', '--install-suggests', '--liveitup', '--llll-----llll--', '--lolwutroflwaffle--', '--master-of-none--', '--mt__', '--nomad--', '--o', '--owo-', '--redacted--', '--verde--', '--xe', '-00000110_00000101-', '-10-5-19-20-5-18-', '-123', '-14k-', '-15gbp', '-2vsU-', '-311-', '-3703', '-3P1C-', '-3point14159-mp', '-4-8-15-16-23-42-', '-420SmokeWeed-', '-480-', '-917-', '-AC-', '-AODH-', '-ASAP-', '-Abbytjie-', '-Abra-Used-Teleport-', '-Academia-', '-AcidBurn-', '-Acyst-', '-Ahab-', '-

### getting user vectors

#### some prototyping

In [11]:
nlp = spacy.load('en_core_web_lg')

In [27]:
test_user[10]

"Don't worry, fam, I only judge from a distance. "

In [28]:
comment = nlp(test_user[10])

In [56]:
comment.tensor.shape[1]

384

In [67]:
([np.sum(nlp(comment).tensor, axis=0) 
          for comment in test_user[:2]][0] + [np.sum(nlp(comment).tensor, axis=0) 
          for comment in test_user[:2]][1]).shape

(384,)

### actual function

In [76]:
def get_repr_tensor(author):
    '''Calculates user's embeddning
    
    Average of all comments tensors (which are average) of all
    comment words. Input's the author which is a list of comments.
    Output is the user embedding.
    '''
    
    n_comments = len(author)
    comment_tensors = [np.sum(nlp(comment).tensor, axis=0)
                       for comment in author]
    repr_tensor = np.sum(comment_tensors, axis=0)
    repr_tensor = repr_tensor / n_comments
          
    return repr_tensor

In [77]:
%%timeit
aa = get_repr_tensor(test_user)

3.75 s ± 443 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
test_tensor = get_repr_tensor(test_user)

In [75]:
test_tensor

array([-2.32087135e+01, -2.69046612e+01, -2.27781582e+00, -1.04774618e+01,
       -3.79487705e+00,  2.17183170e+01, -3.25763798e+00,  9.11980915e+00,
       -1.35382576e+01,  2.22707634e+01, -1.90431671e+01,  1.81136475e+01,
        2.88468647e+01, -1.68553925e+01, -8.22934723e+00,  5.85191584e+00,
        1.68255329e+01,  1.08620729e+01,  1.08128080e+01, -5.33207417e-01,
        2.15015678e+01, -1.20588894e+01,  1.14016018e+01,  3.92791595e+01,
        8.57727051e+00,  8.08818817e+00, -2.04729500e+01, -6.72082233e+00,
        6.78176928e+00, -1.70421171e+00, -1.20518675e+01,  2.21905756e+00,
        1.03622742e+01, -2.56166458e+01, -2.19011366e-01, -1.32845278e+01,
       -2.27031255e+00,  1.59690332e+01,  1.14002771e+01,  1.26823425e+01,
        6.63704205e+00,  1.06936855e+01, -2.13903046e+01,  3.13112183e+01,
       -3.31628299e+00,  1.68966579e+01, -3.26583633e+01,  1.46880550e+01,
       -1.42199841e+01,  1.37701063e+01,  5.40340948e+00,  2.53309488e+00,
        2.53706474e+01, -

### going through all data

In [78]:
def process_users(file_list):
    user_list = []
    for j in range(len(file_list)):
        if j % 10 == 0:
            print(j)
        with open('data/authors/' + file_list[j], 'rb') as f:
            file = pickle.load(f)
            users = file.keys()
            tensors = {user:get_repr_tensor(user) 
                       for user in users}
            user_list.append(tensors)
            
    return user_list

In [ ]:
files[100]

In [ ]:
# the reason for running it in pieces is certainty, not to lose all progress
# if computer crashes, etc.

In [83]:
user_embeddings = process_users(files[:100])
filename = "data/user_embeddings/" + files[0]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [84]:
user_embeddings = process_users(files[100:200])
filename = "data/user_embeddings/" + files[100]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [85]:
user_embeddings = process_users(files[200:300])
filename = "data/user_embeddings/" + files[200]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [86]:
user_embeddings = process_users(files[300:400])
filename = "data/user_embeddings/" + files[300]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [87]:
user_embeddings = process_users(files[400:500])
filename = "data/user_embeddings/" + files[400]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [88]:
user_embeddings = process_users(files[500:600])
filename = "data/user_embeddings/" + files[500]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [89]:
user_embeddings = process_users(files[600:700])
filename = "data/user_embeddings/" + files[600]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [90]:
user_embeddings = process_users(files[700:800])
filename = "data/user_embeddings/" + files[700]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [91]:
user_embeddings = process_users(files[800:900])
filename = "data/user_embeddings/" + files[800]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [92]:
user_embeddings = process_users(files[900:1000])
filename = "data/user_embeddings/" + files[900]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [93]:
user_embeddings = process_users(files[1000:1100])
filename = "data/user_embeddings/" + files[1000]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [94]:
user_embeddings = process_users(files[1100:1200])
filename = "data/user_embeddings/" + files[1100]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [95]:
user_embeddings = process_users(files[1200:1300])
filename = "data/user_embeddings/" + files[1200]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [96]:
user_embeddings = process_users(files[1300:1400])
filename = "data/user_embeddings/" + files[1300]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [97]:
user_embeddings = process_users(files[1400:1500])
filename = "data/user_embeddings/" + files[1400]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [98]:
user_embeddings = process_users(files[1500:1600])
filename = "data/user_embeddings/" + files[1500]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [99]:
user_embeddings = process_users(files[1600:1700])
filename = "data/user_embeddings/" + files[1600]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [100]:
user_embeddings = process_users(files[1700:1800])
filename = "data/user_embeddings/" + files[1700]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [101]:
user_embeddings = process_users(files[1800:1900])
filename = "data/user_embeddings/" + files[1800]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [102]:
user_embeddings = process_users(files[1900:2000])
filename = "data/user_embeddings/" + files[1900]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [103]:
user_embeddings = process_users(files[2000:2100])
filename = "data/user_embeddings/" + files[2000]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [104]:
user_embeddings = process_users(files[2100:2200])
filename = "data/user_embeddings/" + files[2100]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [105]:
user_embeddings = process_users(files[2200:2300])
filename = "data/user_embeddings/" + files[2200]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [106]:
user_embeddings = process_users(files[2300:2400])
filename = "data/user_embeddings/" + files[2300]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [107]:
user_embeddings = process_users(files[2400:2500])
filename = "data/user_embeddings/" + files[2400]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
50
60
70
80
90


In [108]:
user_embeddings = process_users(files[2500:])
filename = "data/user_embeddings/" + files[2500]
fileobj = open(filename, 'wb')
pickle.dump(user_embeddings, fileobj, pickle.HIGHEST_PROTOCOL)
fileobj.close()

0
10
20
30
40
